<a href="https://colab.research.google.com/github/karnamohit/kranka_ucm/blob/master/tf1_test_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing all the useful libraries...

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

Checking the version of TensorFlow...

In [0]:
!pip show tensorflow

Name: tensorflow
Version: 1.14.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: gast, numpy, keras-preprocessing, wrapt, keras-applications, tensorflow-estimator, astor, six, protobuf, termcolor, wheel, tensorboard, absl-py, google-pasta, grpcio
Required-by: stable-baselines, magenta, fancyimpute


Need to import data...

In [0]:
!pwd

/content


Make up data (2x2, sequentially indexed matrices)...

In [3]:
series_data = np.random.rand(100,2,2)      # "time"-series data for a 2x2 evolving matrix with 100 time-steps, randomly initialized
series_data.shape

(100, 2, 2)

Use ```tf.data``` for storing input data (```series_data``` in the example above)

In [4]:
input_slices = tf.data.Dataset.from_tensor_slices(series_data)
next_slice_item = input_slices.make_one_shot_iterator().get_next()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


Run a ```tf.Session()``` (within the context manager) to verify sequential printing of the evolving 2x2 matrix...

In [0]:
with tf.Session() as sess:
  while True:
    try:
      print(sess.run(next_slice_item))
    except tf.errors.OutOfRangeError:
      break

Define batches of the ```input_slices``` (saves memory when building the computational graph)

Set up placeholders for input, true output, predicted output...

In [0]:
x = tf.placeholder(tf.float)